In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import socketserver
import pickle
import urllib.request
import json
from pprint import pprint
from pandas.io.json import json_normalize
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from math import sqrt
import os
import errno
from pymongo import MongoClient
import urllib.parse as urlparse
from influxdb import InfluxDBClient
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize
import csv


In [2]:
def getAllNodeNames(client):
    queryResult = client.query("SHOW TAG VALUES FROM uptime WITH KEY=nodename;")
    nodeNames_temp = list(queryResult.get_points())
    dfnodeNames = pd.DataFrame(nodeNames_temp)
    allNodeNames = dfnodeNames[:]["value"]
    return allNodeNames
def getNamespaceNames(client,node):
    nsQuery = client.query("SHOW TAG VALUES FROM uptime WITH KEY=namespace_name WHERE nodename = '"+node+"';")
    nsQuery_temp = list(nsQuery.get_points())
    dfnsNames = pd.DataFrame(nsQuery_temp)
    allnsNames = dfnsNames[:]["value"]
    return allnsNames
def getAllPodNames(client,node,ns_name):
    queryResult = client.query("SHOW TAG VALUES FROM uptime WITH KEY = pod_name WHERE namespace_name = '"+ns_name+"' AND nodename = '"+node+"';")
    podNames_temp = list(queryResult.get_points())
    dfpodNames = pd.DataFrame(podNames_temp)
    if dfpodNames.empty:
        return dfpodNames
    else:
        allpodNames = dfpodNames[:]["value"]
        return allpodNames
def getCPUUtilizationNode(client, node):
    queryResult = client.query('SELECT * FROM "cpu/node_utilization" where nodename = \''+node+'\' AND type=\'node\';')
    dfcpuUtilization = pd.DataFrame(queryResult['cpu/node_utilization'])
    return dfcpuUtilization
def getCPUUtilizationPod(client, node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "cpu/usage_rate" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfcpuUtilization = pd.DataFrame(queryResult['cpu/usage_rate'])
    return dfcpuUtilization
def getCPUUtilizationPodContainer(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "cpu/usage_rate" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\' AND type=\'pod_container\';')
    dfcpuUtilization = pd.DataFrame(queryResult['cpu/usage_rate'])
    return dfcpuUtilization
def prepareCpuUtilization(client,node,ns_name, pod_name):
    cpuUtilization = getCPUUtilizationNode(client,node)
    podCpuUtilization = getCPUUtilizationPod(client,node,ns_name, pod_name)
    containercpuUtilization = getCPUUtilizationPodContainer(client,node,ns_name, pod_name)
    plt.plot(cpuUtilization.index, cpuUtilization['value'] *1000, 'r', label="node") # plotting t, a separately
    plt.plot(podCpuUtilization.index, podCpuUtilization['value'], 'b', label="pod") # plotting t, b separately
    plt.plot(containercpuUtilization.index, containercpuUtilization['value'], 'g', label="container") # plotting t, c separately
    plt.legend(loc='upper left')
    plt.show()
def getMemoryUtilizationNode(client,node):
    queryResult = client.query('SELECT * FROM "memory/node_utilization" where nodename = \''+node+'\' AND type=\'node\';')
    dfmemUtilization = pd.DataFrame(queryResult['memory/node_utilization'])
    return dfmemUtilization
def getMemoryUtilizationPod(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "memory/usage" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfmemUtilization = pd.DataFrame(queryResult['memory/usage'])
    return dfmemUtilization
def getMemoryUtilizationPodContainer(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "memory/usage" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\' AND type=\'pod_container\';')
    dfmemUtilization = pd.DataFrame(queryResult['memory/usage'])
    return dfmemUtilization
def prepareMemoryUtilization(client,node,ns_name, pod_name):
    memoryUtilization = getMemoryUtilizationNode(client,node)
    podMemoryUtilization = getMemoryUtilizationPod(client,node,ns_name, pod_name)
    containerMemoryUtilization = getMemoryUtilizationPodContainer(client,node,ns_name, pod_name)
    plt.plot(memoryUtilization.index, memoryUtilization['value'], 'r', label="node") # plotting t, a separately
    plt.plot(podMemoryUtilization.index, podMemoryUtilization['value'], 'b', label="pod") # plotting t, b separately
    plt.plot(containerMemoryUtilization.index, containerMemoryUtilization['value'], 'g', label="container") # plotting t, c separately
    plt.legend(loc='upper left')
    plt.show()
def getNetworkTxRatePod(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "network/tx_rate" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfmemUtilization = pd.DataFrame(queryResult['network/tx_rate'])
    return dfmemUtilization
def getNetworkTxPod(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "network/tx" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfmemUtilization = pd.DataFrame(queryResult['network/tx'])
    return dfmemUtilization
def getNetworkTxErrorsPod(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "network/tx_errors" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfmemUtilization = pd.DataFrame(queryResult['network/tx_errors'])
    return dfmemUtilization
def getNetworkTxErrorsRatePod(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "network/tx_errors_rate" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfmemUtilization = pd.DataFrame(queryResult['network/tx_errors_rate'])
    return dfmemUtilization
def prepareNetworkTxRateUtilization(client,node,ns_name, pod_name):
    podNetworTxRate = getNetworkTxRatePod(client,node,ns_name, pod_name)
    podNetworTx = getNetworkTxPod(client,node,ns_name, pod_name)
    podNetworkError = getNetworkTxErrorsPod(client,node,ns_name, pod_name)
    podNetworkErrorRate = getNetworkTxErrorsRatePod(client,node,ns_name, pod_name)
    plt.plot(podNetworTxRate.index, podNetworTxRate['value'], 'b') # plotting t, b separately
    #plt.plot(podNetworTx.index, podNetworTx['value'], 'g') # plotting t, b separately
    #plt.plot(podNetworkError.index, podNetworkError['value'], 'y') # plotting t, b separately
    plt.plot(podNetworkErrorRate.index, podNetworkErrorRate['value'], 'r') # plotting t, b separately
    plt.show()
def getNetworkRxRatePod(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "network/rx_rate" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfmemUtilization = pd.DataFrame(queryResult['network/rx_rate'])
    return dfmemUtilization
def getNetworkRxPod(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "network/rx" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfmemUtilization = pd.DataFrame(queryResult['network/rx'])
    return dfmemUtilization

def getNetworkRxErrorsPod(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "network/rx_errors" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfmemUtilization = pd.DataFrame(queryResult['network/rx_errors'])
    return dfmemUtilization
def getNetworkRxErrorsRatePod(client,node,ns_name, pod_name):
    queryResult = client.query('SELECT * FROM "network/rx_errors_rate" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\';')
    dfmemUtilization = pd.DataFrame(queryResult['network/rx_errors_rate'])
    return dfmemUtilization
def prepareNetworkRxRateUtilization(client,node,ns_name, pod_name):
    podNetworRxRate = getNetworkRxRatePod(client,node,ns_name, pod_name)
    podNetworRx = getNetworkRxPod(client,node,ns_name, pod_name)
    podNetworkError = getNetworkRxErrorsPod(client,node,ns_name, pod_name)
    podNetworkErrorRate = getNetworkRxErrorsRatePod(client,node,ns_name, pod_name)
    plt.plot(podNetworRxRate.index, podNetworRxRate['value'], 'b') # plotting t, b separately
    #plt.plot(podNetworRx.index, podNetworRx['value'], 'g') # plotting t, b separately
    #plt.plot(podNetworkError.index, podNetworkError['value'], 'y') # plotting t, b separately
    plt.plot(podNetworkErrorRate.index, podNetworkErrorRate['value'], 'r') # plotting t, b separately
    plt.show()
def getRelevantNodeName(client,ns_name):
    allNodeNames  = getAllNodeNames(client)
    #nsNames = getNamespaceNames(allNodeNames[0])
    for node in allNodeNames:
        allPodNamesNode = getAllPodNames(client,node,ns_name)
        if(not allPodNamesNode.empty):
            return node

def getNodeResourceUtilizationDf(client, nodeName):
    Result_node_CPU = client.query("SELECT value from \"cpu/node_utilization\" where nodename = '"+nodeName+"' AND type = 'node' ")
    Result_node_MEM = client.query("SELECT value from \"memory/node_utilization\" where nodename = '"+nodeName+"' AND type = 'node' ")

    Result_node_CPU_Cores = client.query("SELECT mean(\"value\") FROM \"cpu/node_capacity\" where nodename = '"+nodeName+
                                         "' AND type = 'node' GROUP BY time(1m)")
    Result_node_mem_node = client.query("SELECT mean(\"value\")FROM \"memory/node_capacity\" where nodename = '"+
                                        nodeName+"' AND type = 'node' GROUP BY time(1m)")

    cpu_points = pd.DataFrame(Result_node_CPU.get_points())
    cpu_points['time'] = pd.to_datetime(cpu_points['time'])
    cpu_points = cpu_points.set_index('time')
    cpu_points.columns = ['node_cpu_util']
    mem_points = pd.DataFrame(Result_node_MEM.get_points())
    mem_points['time'] = pd.to_datetime(mem_points['time'])
    mem_points = mem_points.set_index('time')
    mem_points.columns = ['node_mem_util']

    cores_points = pd.DataFrame(Result_node_CPU_Cores.get_points())
    cores_points['time'] = pd.to_datetime(cores_points['time'])
    cores_points = cores_points.set_index('time')
    cores_points.columns = ['node_cores']

    mem_node_points = pd.DataFrame(Result_node_mem_node.get_points())
    mem_node_points['time'] = pd.to_datetime(mem_node_points['time'])
    mem_node_points = mem_node_points.set_index('time')
    mem_node_points.columns = ['node_mem']

    df_node =pd.concat([cpu_points, mem_points,cores_points,mem_node_points], axis=1)
    return df_node
def getPodResourceUtilizationDf(client, node, ns_name, pod_name):
    Result_Pod_CPU_usage = client.query('SELECT value FROM "cpu/usage_rate" where nodename = \''+node+
                                        '\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+
                                        '\'  AND type=\'pod\';')
    Result_Pod_MEM_usage = client.query('SELECT value from \"memory/usage\" where nodename = \''+node+
                                        '\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+
                                        '\'  AND type=\'pod\';')

    Result_Pod_CPU_limit = client.query('SELECT mean(\"value\") FROM "cpu/limit" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\' group by time(1m);')
    Result_Pod_MEM_limit = client.query('SELECT mean(\"value\") from \"memory/limit\" where nodename = \''+node+
                                        '\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+
                                        '\'  AND type=\'pod\' group by time(1m);')

    Result_Pod_CPU_requests = client.query('SELECT mean(\"value\") FROM "cpu/request" where nodename = \''+node+'\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+'\'  AND type=\'pod\' group by time(1m);')
    Result_Pod_MEM_requests = client.query('SELECT mean(\"value\") from \"memory/request\" where nodename = \''+node+
                                           '\' AND pod_name = \''+pod_name+'\' AND namespace_name = \''+ns_name+
                                           '\'  AND type=\'pod\' group by time(1m);')


    cpu_points_usage = pd.DataFrame(Result_Pod_CPU_usage.get_points())
    cpu_points_usage['time'] = pd.to_datetime(cpu_points_usage['time'])
    cpu_points_usage = cpu_points_usage.set_index('time')
    cpu_points_usage.columns = ['pod_cpu_usage']


    mem_points_usage = pd.DataFrame(Result_Pod_MEM_usage.get_points())
    mem_points_usage['time'] = pd.to_datetime(mem_points_usage['time'])
    mem_points_usage = mem_points_usage.set_index('time')
    mem_points_usage.columns = ['pod_mem_usage']


    cpu_points_limits = pd.DataFrame(Result_Pod_CPU_limit.get_points())
    cpu_points_limits['time'] = pd.to_datetime(cpu_points_limits['time'])
    cpu_points_limits = cpu_points_limits.set_index('time')
    cpu_points_limits.columns = ['pod_cpu_limit']


    mem_points_limits = pd.DataFrame(Result_Pod_MEM_limit.get_points())
    mem_points_limits['time'] = pd.to_datetime(mem_points_limits['time'])
    mem_points_limits = mem_points_limits.set_index('time')
    mem_points_limits.columns = ['pod_mem_limit']


    cpu_points_request = pd.DataFrame(Result_Pod_CPU_requests.get_points())
    cpu_points_request['time'] = pd.to_datetime(cpu_points_request['time'])
    cpu_points_request = cpu_points_request.set_index('time')
    cpu_points_request.columns = ['pod_cpu_request']


    mem_points_request = pd.DataFrame(Result_Pod_MEM_requests.get_points())
    mem_points_request['time'] = pd.to_datetime(mem_points_request['time'])
    mem_points_request = mem_points_request.set_index('time')
    mem_points_request.columns = ['pod_mem_request']

    df_pod =pd.concat([cpu_points_usage, mem_points_usage,cpu_points_limits,mem_points_limits,cpu_points_request,mem_points_request ], axis=1)

    return df_pod
def getRequestsDf(clientK6):
    queryResult = clientK6.query('SELECT sum("value") FROM "vus" group by time(1m);')
    vus = pd.DataFrame(queryResult['vus'])
    vus.columns = ['vus','time']
    vus = vus.set_index('time')


    queryResultReqs = clientK6.query('SELECT sum("value") FROM "http_reqs" group by time(1m);')
    reqs = pd.DataFrame(queryResultReqs['http_reqs'])
    reqs.columns = ['requests','time']
    reqs = reqs.set_index('time')
    queryResultReqsDuration95 = clientK6.query('SELECT percentile("value", 95) FROM "http_req_duration" group by time(1m) ;')
    reqs_duration95 = pd.DataFrame(queryResultReqsDuration95['http_req_duration'])
    reqs_duration95.columns = [ 'requests_duration_percentile_95','time']
    reqs_duration95 = reqs_duration95.set_index('time')
    queryResultReqsDuration90 = clientK6.query('SELECT percentile("value", 90) FROM "http_req_duration" group by time(1m) ;')
    reqs_duration90 = pd.DataFrame(queryResultReqsDuration90['http_req_duration'])
    reqs_duration90.columns = ['requests_duration_percentile_90','time']
    reqs_duration90 = reqs_duration90.set_index('time')

    queryResultMaxDuration = clientK6.query('SELECT max("value") FROM "http_req_duration" group by time(1m);')
    reqs_duration_max = pd.DataFrame(queryResultMaxDuration['http_req_duration'])
    reqs_duration_max.columns = ['requests_duration_max','time']
    reqs_duration_max = reqs_duration_max.set_index('time')

    queryResultMinDuration = clientK6.query('SELECT min("value") FROM "http_req_duration" group by time(1m);')
    reqs_duration_min = pd.DataFrame(queryResultMinDuration['http_req_duration'])
    reqs_duration_min.columns = ['requests_duration_min','time']
    reqs_duration_min = reqs_duration_min.set_index('time')

    queryResultMeanDuration = clientK6.query('SELECT mean("value") FROM "http_req_duration" group by time(1m);')
    reqs_duration_mean = pd.DataFrame(queryResultMeanDuration['http_req_duration'])
    reqs_duration_mean.columns = ['requests_duration_mean','time']
    reqs_duration_mean = reqs_duration_mean.set_index('time')

    queryResultMedianDuration = clientK6.query('SELECT median("value") FROM "http_req_duration" group by time(1m);')
    reqs_duration_median = pd.DataFrame(queryResultMedianDuration['http_req_duration'])
    reqs_duration_median.columns = ['requests_duration_median','time']
    reqs_duration_median = reqs_duration_median.set_index('time')

    finalDF = pd.merge(vus, reqs, left_index=True, right_index=True)
    finalDF = pd.merge(finalDF, reqs_duration95, left_index=True, right_index=True)
    finalDF = pd.merge(finalDF, reqs_duration90, left_index=True, right_index=True)
    finalDF = pd.merge(finalDF,reqs_duration_max, left_index=True, right_index=True)
    finalDF = pd.merge(finalDF,reqs_duration_min, left_index=True, right_index=True)
    finalDF = pd.merge(finalDF,reqs_duration_mean, left_index=True, right_index=True)
    finalDF = pd.merge(finalDF,reqs_duration_median, left_index=True, right_index=True)
    finalDF.index = pd.to_datetime(finalDF.index)

    return finalDF

def getPodsNodesRequestsDf(appNames, client,  clientK6):
    default_ns_name =  "default"
    relevantNodeName = getRelevantNodeName(client,default_ns_name)
    df_pods_node = []
    if relevantNodeName is not None:
        podNames = getAllPodNames(client,relevantNodeName, default_ns_name)
        df_node = getNodeResourceUtilizationDf(client,relevantNodeName)

        for podName in podNames:
            if appNames[0] in podName:
                df_pod = getPodResourceUtilizationDf(client,relevantNodeName, default_ns_name, podName)
                finalDF = pd.merge(df_node,df_pod, left_index=True, right_index=True)
                requestsDF = getRequestsDf(clientK6)
                finalDF = pd.merge(finalDF,requestsDF, left_index=True, right_index=True)
                if(finalDF['pod_cpu_limit'].values[0]==0):
                    finalDF['pod_cpu_usage'] = finalDF['pod_cpu_usage']/(finalDF['node_cores'])
                    finalDF['pod_cpu_limit'] = finalDF['node_cores']/1000
                    finalDF['pod_cpu_request'] = finalDF['node_cores']/1000
                else:
                    finalDF['pod_cpu_usage'] = finalDF['pod_cpu_usage']/(finalDF['pod_cpu_limit'])
                    finalDF['pod_cpu_limit'] = finalDF['pod_cpu_limit']/1000
                    finalDF['pod_cpu_request'] = finalDF['pod_cpu_request']/1000

                if(finalDF['pod_mem_limit'].values[0]==0):
                    finalDF['pod_mem_usage'] = finalDF['pod_mem_usage']/(finalDF['node_mem'])
                    finalDF['pod_mem_limit'] = finalDF['node_mem']/(1073741824)
                    finalDF['pod_mem_request'] = finalDF['node_mem']/(1073741824)
                else:
                    finalDF['pod_mem_usage'] = finalDF['pod_mem_usage']/(finalDF['pod_mem_limit'])
                    finalDF['pod_mem_limit'] = finalDF['pod_mem_limit']/(1073741824)
                    finalDF['pod_mem_request'] = finalDF['pod_mem_request']/(1073741824)
                finalDF['node_cores'] = finalDF['node_cores']/1000
                finalDF['node_mem'] = finalDF['node_mem']/(1073741824)

                finalDF = finalDF.fillna(0)
                finalDF = finalDF[(finalDF.T != 0).any()]
                df_pods_node.append(finalDF)
    return df_pods_node

def getAndCombineAllDbs( host, port, username, password,appNames, folderNames):
    allFinalDFs = []
    print("FolderNames len = ", len(folderNames))
    for folderName in folderNames:
        client = InfluxDBClient(host, port,username , password, folderName+'_k8s')
        clientK6 = InfluxDBClient(host, port, username, password, folderName+'_TestK6')
        df_pods_node = getPodsNodesRequestsDf(appNames, client, clientK6)
        print(folderName)
        if(len(df_pods_node)>0):
            finalDF = pd.DataFrame()
            finalDF['pod_util_cpu_sum'] = 0
            finalDF['pod_util_mem_sum'] = 0
            first = 1
            for i in range(len(df_pods_node)):
                df_pods_node[i] = df_pods_node[i].reset_index(drop=True)
                if(first==1):
                    finalDF['pod_util_cpu_sum'] = df_pods_node[i]['pod_cpu_usage']
                    finalDF['pod_util_mem_sum'] = df_pods_node[i]['pod_mem_usage']
                    first=0
                else:
                    finalDF['pod_util_cpu_sum'] = finalDF['pod_util_cpu_sum'] +  df_pods_node[i]['pod_cpu_usage']
                    finalDF['pod_util_mem_sum'] = finalDF['pod_util_mem_sum'] +  df_pods_node[i]['pod_mem_usage']

            finalDF['num_pods'] = int(len(df_pods_node))
            finalDF['pod_util_cpu_avg'] = finalDF['pod_util_cpu_sum']/finalDF['num_pods']
            finalDF['pod_util_mem_avg'] = finalDF['pod_util_mem_sum']/finalDF['num_pods']

            finalDF = pd.concat([finalDF, df_pods_node[0][['node_cores', 'node_mem','node_cpu_util','node_mem_util', 'pod_cpu_limit', 'pod_cpu_request','pod_mem_limit',
                                                           'pod_mem_request','vus','requests','requests_duration_percentile_95',
                                                           'requests_duration_percentile_90','requests_duration_max', 'requests_duration_min',
                                                           'requests_duration_mean', 'requests_duration_median'
                                                           ]]], axis=1)

            allFinalDFs.append(finalDF)
    df = pd.DataFrame()
    print("All Dfs len = ", len(allFinalDFs))
    for idx in range(len(allFinalDFs)):
        df = df.append(allFinalDFs[idx])

    final_df  = df[['requests','requests_duration_mean','num_pods','pod_cpu_limit','node_cores', 'node_mem','pod_mem_limit','pod_util_cpu_avg','pod_util_mem_avg',
                    ]]
    final_df['pod_util_cpu_avg'] = final_df['pod_util_cpu_avg']*final_df['pod_cpu_limit']
    final_df['pod_util_mem_avg'] = final_df['pod_util_mem_avg']*final_df['pod_mem_limit']
    final_df = final_df.sort_values(['requests'])
    final_df = final_df[(final_df[['pod_util_cpu_avg','pod_util_mem_avg','requests_duration_mean']] != 0).all(axis=1)]
    final_df = final_df[np.isfinite(final_df['requests'])]
    final_df = final_df[np.isfinite(final_df['requests_duration_mean'])]
    final_df = final_df[np.isfinite(final_df['pod_util_cpu_avg'])]
    final_df = final_df[np.isfinite(final_df['pod_util_mem_avg'])]
    final_df = final_df[final_df.requests_duration_mean < 2500]
    final_df = final_df.reset_index(drop=True)
    return final_df

def train_and_return_model(host, port, username, password,appType, appNames, folderNames ):
    df = getAndCombineAllDbs(host, port, username, password,appNames, folderNames)
    df = df.sort_values(['pod_util_cpu_avg'])
    df_X = df[['pod_util_cpu_avg','pod_cpu_limit','pod_mem_limit', 'num_pods', 'requests_duration_mean']].values
    df_Y = df[['requests']].values

    X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33, random_state=42)
    polybig_features = PolynomialFeatures(degree=2, include_bias=False)
    std_scaler = StandardScaler()
    lin_reg = linear_model.LinearRegression()
    regr = Pipeline([
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    # The coefficients

    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))

    #print ('Test score %.2f', regr.score(X_test, y_test) )
    print("Train Mean squared error: %.2f"
          % mean_squared_error(y_train, regr.predict(X_train)))
    rms = sqrt(mean_squared_error(y_test, y_pred))
    print('RMs score: %.2f' % rms)
    return regr, rms
def train_and_return_model_replicas(host, port, username, password,appType, appNames, folderNames ):
    df = getAndCombineAllDbs(host, port, username, password,appNames, folderNames)
    df = df.sort_values(['pod_util_cpu_avg'])
    df_X = df[['pod_util_cpu_avg','pod_cpu_limit','pod_mem_limit', 'requests', 'requests_duration_mean']].values
    df_Y = df[['num_pods']].values
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33, random_state=42)
    polybig_features = PolynomialFeatures(degree=2, include_bias=False)
    std_scaler = StandardScaler()
    lin_reg = linear_model.LinearRegression()
    regr = Pipeline([
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    # The coefficients

    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))

    #print ('Test score %.2f', regr.score(X_test, y_test) )
    print("Train Mean squared error: %.2f"
          % mean_squared_error(y_train, regr.predict(X_train)))
    rms = sqrt(mean_squared_error(y_test, y_pred))
    print('RMs score: %.2f' % rms)
    return regr, rms

def train_and_return_model_smart(host, port, username, password,appType, appNames, folderNames ):
    df = getAndCombineAllDbs(host, port, username, password,appNames, folderNames)
    df = df.sort_values(['pod_util_cpu_avg'])
    df = df.head(50)
    if(appType == "compute"):
        df_X = df[['pod_util_cpu_avg','requests_duration_mean']].values
    else:
        df_X = df[['pod_util_cpu_avg','requests_duration_mean']].values

    df_Y = df[['requests']].values
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33, random_state=42)
    polybig_features = PolynomialFeatures(degree=2, include_bias=False)
    std_scaler = StandardScaler()
    lin_reg = linear_model.LinearRegression()
    regr = Pipeline([
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    # The coefficients

    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(y_test, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_pred))

    #print ('Test score %.2f', regr.score(X_test, y_test) )
    print("Train Mean squared error: %.2f"
          % mean_squared_error(y_train, regr.predict(X_train)))
    rms = sqrt(mean_squared_error(y_test, y_pred))
    print('RMs score: %.2f' % rms)
    return regr, rms
def do_training( host, port, username, password, instanceFamily, appNames, appType, folderNames, filename):
    model, rms = train_and_return_model(host, port, username, password,appType, appNames,folderNames)
    print(filename)
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    pickle.dump(model, open(filename, 'wb'))
    return rms
def do_training_replicas( host, port, username, password, instanceFamily, appNames, appType, folderNames, filename):
    model, rms = train_and_return_model_replicas(host, port, username, password,appType, appNames,folderNames)
    print(filename)
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    pickle.dump(model, open(filename, 'wb'))
    return rms

def do_training_smart( host, port, username, password, instanceFamily, appNames, appType, folderNames, filename):
    model, rms = train_and_return_model_smart(host, port, username, password,appType, appNames,folderNames)
    print(filename)
    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    pickle.dump(model, open(filename, 'wb'))
    return rms
def get_folder_names( appName, colName, dbName):
    mongoclient = MongoClient(mongo_host, 27017, username=mongo_username, password=mongo_password)
    db = mongoclient[dbName]
    col = db[colName]
    datapoints = list(col.find({"servicename": appName}))
    dfMongo = json_normalize(datapoints)
    return dfMongo.foldername


In [3]:
appNamesConst = ['primeapp', 'movieapp', 'webacapp', 'serveapp', 'mixalapp']
appTypesConst = ['compute', 'dbaccess', 'web', 'sandbox', 'mix']
mainServiceNamesConst =[[0 for x in range(5)] for y in range(5)]
mainServiceNamesConst[0] = ['primeapp']
mainServiceNamesConst[1] = ['movieapp']
mainServiceNamesConst[2] = ['webacapp']
mainServiceNamesConst[3] = ['serveapp']
mainServiceNamesConst[4] = ['primeapp', 'movieapp', 'webacapp', 'serveapp']
appTest = 0
appName = appNamesConst[appTest]
appType = appTypesConst[appTest]
instanceFamily = "t2"
mainServiceName = mainServiceNamesConst[appTest][0]
requestDuration = 1000

host = '141.40.254.24'
port = '8086'
username= 'root'
password = 'root'

mongo_host = '141.40.254.24'
mongo_port = 27017
mongo_username= 'mongoDBSecureUser12378'
mongo_password = 'youdonthavetoknowthis8998'

colName = "ALL_BRUTE_FORCE_CONDUCTED_TEST_NAMES"
dbName = "TERMINUS"

In [4]:
print (appName, ", ", mainServiceName, ", ", appType)

primeapp ,  primeapp ,  compute


In [5]:
folderNamesPrimeapp = get_folder_names(appNamesConst[0], colName, dbName)
folderNamesMovieapp = get_folder_names(appNamesConst[1], colName, dbName)
folderNamesWebapp = get_folder_names(appNamesConst[2], colName, dbName)
folderNamesMixapp = get_folder_names(appNamesConst[3], colName, dbName)

ServerSelectionTimeoutError: 141.40.254.24:27017: timed out

In [ ]:
folderNames = folderNamesPrimeapp
appNames = [mainServiceName]

In [ ]:
dfApp = getAndCombineAllDbs(host, port, username, password,appNames, folderNames)
appName = appName+ "_"+ mainServiceName + ".csv"
dfApp.to_csv(appName)



In [26]:
folderNames = folderNamesMovieapp
appNames = [mainServiceNamesConst[1][0]]
dfAppMovie = getAndCombineAllDbs(host, port, username, password,appNames, folderNames)
appNameMovie = appNamesConst[1]+ "_"+ mainServiceNamesConst[1][0] + ".csv"


FolderNames len =  61
sv1t1rc1nc1t2microdbaccessmovieappfalseloya
sv1t1rc1nc1t2nanodbaccessmovieappfalsezoss
sv1t1rc1nc1t2smalldbaccessmovieappfalsec2tp
s1t1rc1nc1t2mediumdbaccessmovieappt2microy64
s1t1rc1nc1t2mediumdbaccessmovieappt2nanomtp
s1t1rc1nc1t2mediumdbaccessmovieappt2small3rz
sv1t1rc1nc1t2mediumdbaccessmovieappfalsedv3acs
s1t1rc1nc1t2largedbaccessmovieappt2micro8uf
s1t1rc1nc1t2largedbaccessmovieappt2nanofch
s1t1rc1nc1t2largedbaccessmovieappt2smallhs9
s1t1rc1nc1t2xlargedbaccessmovieappt2smallpn8
sv1t1rc1nc1t2largedbaccessmovieappfalsee1hics
s1t1rc1nc1t2xlargedbaccessmovieappt2nanor1n
s1t1rc1nc1t2xlargedbaccessmovieappt2mediumdvw
s1t1rc1nc1t2xlargedbaccessmovieappt2largejn8
s1t1rc1nc1t2xlargedbaccessmovieappt2micro8zu
s1t1rc2nc1t2xlargedbaccessmovieapp500r9z
s1t1rc1nc1t2xlargedbaccessmovieappt2mediumq4q
s1t1rc3nc1t2xlargedbaccessmovieapp100k2g
s1t1rc3nc1t2xlargedbaccessmovieapp500hjl
s1t1rc1nc1t2xlargedbaccessmovieapp5003ba
s1t1rc1nc1t2xlargedbaccessmovieappt2smallosm
s1t1rc3nc

C:\Users\ansjin\Anaconda\lib\site-packages\ipykernel_launcher.py:343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ansjin\Anaconda\lib\site-packages\ipykernel_launcher.py:344: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


AttributeError: 'str' object has no attribute 'to_csv'

In [27]:
dfAppMovie.to_csv(appNameMovie)

In [11]:
appName = appName+ "_"+ mainServiceName + ".csv"
df = pd.read_csv(appName, header=0, index_col=0)

In [12]:
df.head()

,requests,requests_duration_mean,num_pods,pod_cpu_limit,node_cores,node_mem,pod_mem_limit,pod_util_cpu_avg,pod_util_mem_avg
0,13.0,128.577375,2,0.5,4.0,15.674213,0.488281,0.4500,0.080709
1,44.0,16.816887,1,1.0,1.0,1.955933,1.955933,0.0030,0.026791
2,45.0,16.453179,2,1.0,4.0,15.674213,0.976562,0.0015,0.019331
3,45.0,15.507512,1,1.0,4.0,15.674213,0.976562,0.0040,0.021217
4,47.0,18.539949,2,0.5,2.0,7.799198,0.488281,0.0010,0.018446


In [ ]:
#brute-force-training

appNames = [mainServiceName]
df = df.sort_values(['pod_util_cpu_avg'])
df_X = df[['pod_util_cpu_avg','pod_cpu_limit','pod_mem_limit', 'num_pods', 'requests_duration_mean']].values
df_Y = df[['requests']].values
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33, random_state=42)
polybig_features = PolynomialFeatures(degree=3, include_bias=False)
std_scaler = StandardScaler()
lin_reg = linear_model.LinearRegression()
regr = Pipeline([
    ("poly_features", polybig_features),
    ("std_scaler", std_scaler),
    ("lin_reg", lin_reg),
])
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

#print ('Test score %.2f', regr.score(X_test, y_test) )
print("Train Mean squared error: %.2f"
      % mean_squared_error(y_train, regr.predict(X_train)))
rms = sqrt(mean_squared_error(y_test, y_pred))
print('RMs score: %.2f' % rms)

In [ ]:
from sklearn.svm import SVR
appNames = [mainServiceName]
df = df.sort_values(['pod_util_cpu_avg'])
df_X = df[['pod_util_cpu_avg','pod_cpu_limit','pod_mem_limit', 'num_pods']].values
df_Y = df[['requests']].values
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33, random_state=42)

svr_rbf = SVR(kernel= 'rbf', C= 1e3, gamma= 0.1) # defining the support vector regression models
svr_lin = SVR(kernel= 'linear', C= 1e3)
svr_poly = SVR(kernel= 'poly', C= 1e3, degree= 2)
svr_rbf.fit(X_train, y_train) # fitting the data points in the models
svr_lin.fit(X_train, y_train)
svr_poly.fit(X_train, y_train)

y_predsvr_rbf = svr_rbf.predict(X_test)
y_predsvr_lin = svr_lin.predict(X_test)
y_predsvr_poly = svr_poly.predict(X_test)

rms = sqrt(mean_squared_error(y_test, y_predsvr_rbf))
print('RMs score: %.2f' % rms)

rms = sqrt(mean_squared_error(y_test, y_predsvr_lin))
print('RMs score: %.2f' % rms)

rms = sqrt(mean_squared_error(y_test, y_predsvr_poly))
print('RMs score: %.2f' % rms)

In [ ]:
from sklearn.svm import SVR
appNames = [mainServiceName]
df = df.sort_values(['pod_util_cpu_avg'])
df_X = df[['pod_util_cpu_avg','pod_cpu_limit','pod_mem_limit', 'num_pods', 'requests_duration_mean']].values
df_Y = df[['requests']].values
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33, random_state=42)

svr_rbf = SVR(kernel= 'rbf', C= 1e3, gamma= 0.1) # defining the support vector regression models
svr_lin = SVR(kernel= 'linear', C= 1e3)
svr_poly = SVR(kernel= 'poly', C= 1e3, degree= 2)
svr_rbf.fit(X_train, y_train) # fitting the data points in the models
svr_lin.fit(X_train, y_train)
svr_poly.fit(X_train, y_train)

y_predsvr_rbf = svr_rbf.predict(X_test)
y_predsvr_lin = svr_lin.predict(X_test)
y_predsvr_poly = svr_poly.predict(X_test)

rms = sqrt(mean_squared_error(y_test, y_predsvr_rbf))
print('RMs score: %.2f' % rms)

rms = sqrt(mean_squared_error(y_test, y_predsvr_lin))
print('RMs score: %.2f' % rms)

rms = sqrt(mean_squared_error(y_test, y_predsvr_poly))
print('RMs score: %.2f' % rms)

C:\Users\ansjin\Anaconda\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
#train-replicas

filename = "/app/training/preTrainedReplicas/"+appType+"/"+appName+"/"+mainServiceName+"/"+instanceFamily+"/"+"trained.sav"
appNames = [mainServiceName]
df = df.sort_values(['pod_util_cpu_avg'])
df_X = df[['pod_util_cpu_avg','pod_cpu_limit','pod_mem_limit', 'requests', 'requests_duration_mean']].values
df_Y = df[['num_pods']].values
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33, random_state=42)
polybig_features = PolynomialFeatures(degree=2, include_bias=False)
std_scaler = StandardScaler()
lin_reg = linear_model.LinearRegression()
regr = Pipeline([
    ("poly_features", polybig_features),
    ("std_scaler", std_scaler),
    ("lin_reg", lin_reg),
])
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

#print ('Test score %.2f', regr.score(X_test, y_test) )
print("Train Mean squared error: %.2f"
      % mean_squared_error(y_train, regr.predict(X_train)))
rms = sqrt(mean_squared_error(y_test, y_pred))
print('RMs score: %.2f' % rms)
return regr, rms

Mean squared error: 0.17
Variance score: 0.79
Train Mean squared error: 0.17
RMs score: 0.41


SyntaxError: 'return' outside function (<ipython-input-34-94e85addf0ee>, line 35)

In [ ]:
#brute-force-training-prediction
numcoresUtil = urlparse.parse_qs(parsed.query)['numcoresutil'][0]
numcoresLimit = urlparse.parse_qs(parsed.query)['numcoreslimit'][0]
nummemLimit = urlparse.parse_qs(parsed.query)['nummemlimit'][0]
replicas = 1

filename = "/app/training/preTrained/"+appType+"/"+appName+"/"+mainServiceName+"/"+instanceFamily+"/"+"trained.sav"
print(filename)
loaded_model = pickle.load(open(filename, 'rb'))
val = [[float(numcoresUtil),float(numcoresLimit),float(nummemLimit),float(replicas),float(requestDuration)]]
predict = loaded_model.predict(val)
output=""+str(predict[0][0])
print (predict)
print (output)

In [ ]:
#train-replicas-prediction 
numcoresUtil = urlparse.parse_qs(parsed.query)['numcoresutil'][0]
numcoresLimit = urlparse.parse_qs(parsed.query)['numcoreslimit'][0]
nummemLimit = urlparse.parse_qs(parsed.query)['nummemlimit'][0]
msc = urlparse.parse_qs(parsed.query)['msc'][0]   
filename = "/app/training/preTrainedReplicas/"+appType+"/"+appName+"/"+mainServiceName+"/"+instanceFamily+"/"+"trained.sav"
print(filename)
loaded_model = pickle.load(open(filename, 'rb'))
val = [[float(numcoresUtil),float(numcoresLimit),float(nummemLimit),float(msc),float(requestDuration)]]
predict = loaded_model.predict(val)
output=""+str(predict[0][0])
print (predict)
print (output)
            

In [ ]:
#smart-test-train 
folderName = urlparse.parse_qs(parsed.query)['containerName'][0]
appNames = [mainServiceName]
folderNames = [folderName]
filename = "/app/training/smartTest/"+appType+"/"+appName+"/"+mainServiceName+"/"+instanceFamily+"/"+folderName+".sav"
df = df.sort_values(['pod_util_cpu_avg'])
df = df.head(50)
if(appType == "compute"):
    df_X = df[['pod_util_cpu_avg','requests_duration_mean']].values
else:
    df_X = df[['pod_util_cpu_avg','requests_duration_mean']].values

df_Y = df[['requests']].values
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.33, random_state=42)
polybig_features = PolynomialFeatures(degree=2, include_bias=False)
std_scaler = StandardScaler()
lin_reg = linear_model.LinearRegression()
regr = Pipeline([
    ("poly_features", polybig_features),
    ("std_scaler", std_scaler),
    ("lin_reg", lin_reg),
])
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

#print ('Test score %.2f', regr.score(X_test, y_test) )
print("Train Mean squared error: %.2f"
      % mean_squared_error(y_train, regr.predict(X_train)))
rms = sqrt(mean_squared_error(y_test, y_pred))
print('RMs score: %.2f' % rms)


In [ ]:
#smart-test-train prediction
numcoresUtil = urlparse.parse_qs(parsed.query)['numcoresutil'][0]
numcoresLimit = urlparse.parse_qs(parsed.query)['numcoreslimit'][0]
nummemLimit = urlparse.parse_qs(parsed.query)['nummemlimit'][0]
folderName = urlparse.parse_qs(parsed.query)['containerName'][0]
filename = "/app/training/smartTest/"+appType+"/"+appName+"/"+mainServiceName+"/"+instanceFamily+"/"+folderName+".sav"
loaded_model = pickle.load(open(filename, 'rb'))
val = [[]]
if(appType=="compute"):
    val = [[float(numcoresUtil),float(requestDuration)]]
else:
    val = [[float(numcoresUtil),float(requestDuration)]]

predict = loaded_model.predict(val)
output=""+str(predict[0][0])
print (predict)
print (output)


In [ ]:
 #actual trn
folderName = urlparse.parse_qs(parsed.query)['containerName'][0]
appNames = [mainServiceName]
folderNames = [folderName]
df = getAndCombineAllDbs(host, port, username, password,appNames, folderNames)
actualValDf= df.sort_values(['requests'])
final_df = actualValDf[actualValDf.requests_duration_mean < float(requestDuration)]
val = float(final_df.tail(1).requests)
output=""+str(val)
print (val)
print (output)